This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [30]:
import json

# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = json.load(open("./api_key.json"))['api_key']

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [25]:
# First, import the relevant modules

import requests

In [33]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
base_url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json"
parameters = {'api_key': API_KEY, 'limit': '1'}
r = requests.get(base_url, params = parameters)

In [34]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [35]:
# First we should call the dataset with updated parameters
parameters = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}
r = requests.get(base_url, params = parameters)
zeiss_data_dict = r.json()
type(zeiss_data_dict)

dict

The AFX_X stock data is imported to the project and the response is converted to a data dictionary. First, I want to investigate the dictionary.

In [36]:
zeiss_data_dict['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

The dictionary has nested lists for column names and data. I need the column names to identify the index of each parameter in the data list.

In [37]:
zeiss_data_dict['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

I create new lists to extract the desired values and find the max and mins.

In [38]:
opening_price = []
closing_price = []
tarding_vol = []
dailypricechange = []

for dailydata in zeiss_data_dict['dataset_data']['data']:
    # some daily opening data is missing. This filter exclude any days when the opening price is None.
    if dailydata[1] is not None:
        opening_price.append(dailydata[1])
        closing_price.append(dailydata[4])
        tarding_vol.append(dailydata[6])
        dailypricechange.append(round(dailydata[2]-dailydata[3],2))

closingpricechange = []
for i in range(len(closing_price)-1):
    closingpricechange.append(round(closing_price[i+1]-closing_price[i], 2))

Let's print the answers for questions 3, 4 and 5.

In [39]:
print ('Q3. The highest opening price is:', max(opening_price), 'and the lowest opening price is:', min(opening_price))
print('Q4. The largest change in any one day (based on High and Low price) is:', max(dailypricechange))
print('Q5. The largest change between any two days (based on Closing Price) is:', max(closingpricechange))

Q3. The highest opening price is: 53.11 and the lowest opening price is: 34.0
Q4. The largest change in any one day (based on High and Low price) is: 2.81
Q5. The largest change between any two days (based on Closing Price) is: 2.56


I need to calculate the mean and meadian for daily trading volumes to answer questions 6 and 7.

In [40]:
mean = round(sum(tarding_vol)/len(tarding_vol),2)

tarding_vol.sort()
if (len(tarding_vol) % 2) == 0:
    # if number of records are even the median is the mean of the two middlemost values.
    median = (tarding_vol[int(len(tarding_vol) / 2)]+tarding_vol[int(len(tarding_vol) / 2)+1])/2
else:
    median = trading_vol[int(len(tarding_vol) / 2) + 1]

print('Q6. The average daily trading volume during this year is:', mean)
print('Q7. The median trading volume during this year is:', median)

Q6. The average daily trading volume during this year is: 89140.98
Q7. The median trading volume during this year is: 75630.5
